<a href="https://colab.research.google.com/github/workingbetter/ITNPBD5_Dissertation/blob/main/draft2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import random
import pandas as pd
import numpy as np
from deap import base, creator, tools, algorithms

# Load charging rate data
charging_rate_df = pd.read_excel('/content/drive/MyDrive/ITNPBD5/dataset/charging_curve_data.xlsx')

# Load charging station location data
station_df = pd.read_excel('/content/drive/MyDrive/ITNPBD5/dataset/charging_station_location_data.xlsx')

# Assigning given values
initial_soc = 0.4  # initial state of charge
total_distance = 500  # total travel distance in miles
average_speed = 50  # average speed in mph
discharge_rate = 0.004  # discharge rate in SOC per mile
min_soc = 0.09  # minimum SOC
min_charging_time = 15  # minimum charging time in minutes


# Create the fitness and individual classes
if "FitnessMin" in creator.__dict__:
    del creator.__dict__["FitnessMin"]
creator.create('FitnessMin', base.Fitness, weights=(-1.0, -1.0, -1.0,))

creator.create('StationDecision', list)
creator.create('Individual', creator.StationDecision, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register('stop_or_not', random.randint, 0, 1)
toolbox.register('charge_to', random.uniform, min_soc, 1)
toolbox.register('station_decision', tools.initCycle, creator.StationDecision, (toolbox.stop_or_not, toolbox.charge_to), n=1)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.station_decision, n=len(station_df))
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

# Define the evaluation function
def evaluate(individual):
    total_time = 0
    total_charging_time = 0
    total_driving_time = 0
    soc = initial_soc
    previous_location = 0
    charging_times = []
    socs_before_charging = []
    socs_after_charging = []

    for i, decision in enumerate(individual):
        stop, charge_to = decision

        # Driving time to next station
        distance = station_df.loc[i, 'location(mile)'] - previous_location
        driving_time = (distance / average_speed) * 60  # Convert hours to minutes
        total_time += driving_time
        total_driving_time += driving_time  # Added this line

        # Update SOC and location
        soc -= distance * discharge_rate
        previous_location = station_df.loc[i, 'location(mile)']

        if stop:
            # Check if the SOC is enough to reach the station
            if soc < min_soc:
                return float('inf'), float('inf'), float('inf')  # Return very high fitness values

            # Charging
            socs_before_charging.append(soc)
            charging_time = 0
            while soc < charge_to and charging_time < len(charging_rate_df):
                soc = min(charge_to, charging_rate_df.loc[charging_time, 'state_of_charge'] / 100)
                charging_time += 1
            # Ensure at least minimum charging time
            if charging_time < min_charging_time:
                charging_time = min_charging_time
            total_time += charging_time
            total_charging_time += charging_time  # Added this line
            charging_times.append(charging_time)
            socs_after_charging.append(soc)

    # Add the final leg of the journey
    final_distance = total_distance - previous_location
    final_driving_time = (final_distance / average_speed) * 60
    total_time += final_driving_time
    total_driving_time += final_driving_time  # Added this line

    # Check if the SOC is enough to reach the destination
    if soc < min_soc:
        return float('inf'), float('inf'), float('inf')  # Return very high fitness values

    individual.charging_times = charging_times
    individual.socs_before_charging = socs_before_charging
    individual.socs_after_charging = socs_after_charging

    return total_time, total_charging_time, total_driving_time  # Now it returns three values






# Define the genetic operators
toolbox.register('evaluate', evaluate)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb=0.05)
toolbox.register('select', tools.selTournament, tournsize=3)

# Define the main function
def main():
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register('Avg', np.mean)
    stats.register('Min', np.min)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, stats=stats, halloffame=hof, verbose=True)

    best_individual = hof[0]
    fitness = best_individual.fitness.values[0]

    return pop, log, hof


# Run the GA
pop, log, hof = main()

# Print charging times and socs before and after charging at each station
best_individual = hof[0]
print(f'Charging times: {best_individual.charging_times}')
print(f'SOCs before charging: {best_individual.socs_before_charging}')
print(f'SOCs after charging: {best_individual.socs_after_charging}')



total_travel_time_mins = best_individual.fitness.values[0]
total_charging_time_mins = best_individual.fitness.values[1]
total_driving_time_mins = best_individual.fitness.values[2]

total_travel_time = f'{int(total_travel_time_mins // 60)} hours {int(total_travel_time_mins % 60)} minutes'
total_charging_time = f'{int(total_charging_time_mins // 60)} hours {int(total_charging_time_mins % 60)} minutes'
total_driving_time = f'{int(total_driving_time_mins // 60)} hours {int(total_driving_time_mins % 60)} minutes'

# Print the total travel time, charging time, and driving time
print(f'Total travel time: {total_travel_time}')
print(f'Total charging time: {total_charging_time}')
print(f'Total driving time: {total_driving_time}')


# Print the route
print('Route:')
previous_location = 0
last_charging_station = 0
charging_time_index = 0
for i, decision in enumerate(hof[0]):
    stop, charge_to = decision
    if stop:
        print(f'From station {last_charging_station} to charging station {i+1} ({(station_df.loc[i, "location(mile)"] - previous_location) / average_speed * 60} minutes)')
        print(f'Charging at station {i+1} for {best_individual.charging_times[charging_time_index]} minutes (from {best_individual.socs_before_charging[charging_time_index]} to {best_individual.socs_after_charging[charging_time_index]})')
        last_charging_station = i+1
        previous_location = station_df.loc[i, 'location(mile)']
        charging_time_index += 1

print(f'From charging station {last_charging_station} to destination ({(total_distance - previous_location) / average_speed * 60} minutes)')


/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'StationDecision' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	Avg    	Min
0  	100   	1297.29	600
1  	61    	1215.85	600
2  	61    	1155.01	600
3  	50    	1110.82	600
4  	61    	1074.79	600
5  	61    	1048.72	600
6  	67    	1027.05	600
7  	50    	1004.36	600
8  	69    	980.9  	600
9  	61    	960.28 	600
10 	56    	939.253	600
11 	50    	912.373	600
12 	68    	892.087	600
13 	71    	870.5  	595
14 	55    	843.733	525
15 	69    	820.46 	508
16 	66    	799.213	508
17 	68    	777.067	471
18 	63    	756.5  	471
19 	65    	inf    	459
20 	68    	726.693	442
21 	57    	712.84 	409
22 	61    	inf    	355
23 	62    	inf    	370
24 	58    	inf    	355
25 	67    	inf    	328
26 	67    	inf    	320
27 	53    	inf    	302
28 	65    	inf    	276
29 	69    	inf    	266
30 	61    	inf    	232
31 	71    	inf    	211
32 	59    	inf    	206
33 	54    	inf    	206
34 	59    	inf    	194
35 	66    	inf    	171
36 	51    	inf    	153
37 	66    	inf    	153
38 	56    	inf    	136
39 	57    	inf    	136
40 	69    	inf    	95 
Charging times: [15, 21, 23, 21, 1